In [12]:
import sqlite3

def get_table_name(database_file):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()

    # Execute PRAGMA statement to get table information
    cursor.execute("PRAGMA table_info(thegate);")  # Replace 'your_table' with the actual table name
    table_info = cursor.fetchall()

    # Extract the table name from the first row
    if table_info:
        table_name = table_info[0][1]
        print(f'Table Name: {table_name}')
    else:
        print('Table not found.')

    # Close the database connection
    conn.close()

# Example usage
database_file = 'thegate.db'  # Replace with your database file
get_table_name(database_file)


Table not found.


In [14]:
# Don't use
import sqlite3
import xml.etree.ElementTree as ET

# Function to convert SQLite data to XML
def convert_sqlite_to_xml(database_file, output_xml_file, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()

    # Execute a query to fetch data from the specified table
    cursor.execute(f'SELECT * FROM {table_name}')
    rows = cursor.fetchall()

    # Create XML structure
    root = ET.Element('root')
    for row in rows:
        element = ET.SubElement(root, 'row')
        for i, column_name in enumerate(cursor.description):
            field = ET.SubElement(element, column_name[0])
            field.text = str(row[i])

    # Create an ElementTree object and write to an XML file
    tree = ET.ElementTree(root)
    tree.write(output_xml_file)

    # Close the database connection
    conn.close()

# Example usage
database_file = 'thegate.db'  # Replace with your database file
output_xml_file = 'output.xml'  # Specify the output XML file
table_name = 'gate_articles'  # Replace with your table name (e.g., 'your_table')

convert_sqlite_to_xml(database_file, output_xml_file, table_name)


In [15]:
# Don't use
import sqlite3
import xml.etree.ElementTree as ET
from xml.dom import minidom

def convert_sqlite_to_wordpress_xml(database_file, output_xml_file, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()

    # Execute a query to fetch data from the specified table
    cursor.execute(f'SELECT * FROM {table_name}')
    rows = cursor.fetchall()

    # Create WordPress XML structure
    root = ET.Element('rss', version="2.0")
    channel = ET.SubElement(root, 'channel')

    # Add WordPress channel elements (title, link, description, etc.)
    channel_elements = {
        'title': 'The Gate',
        'link': 'thegateuchicago.wordpress.com',
        'description': 'Your WordPress Site Description',
        'pubDate': 'Thu, 25 Jan 2024 23:20:26 +0000',  # Replace with your date
        'language': 'en',
        'wp:wxr_version': '1.2',
        'wp:base_site_url': 'http://wordpress.com/',
        'wp:base_blog_url': 'thegateuchicago.wordpress.com',
    }

    for element_name, element_text in channel_elements.items():
        ET.SubElement(channel, element_name).text = element_text

    # Add WordPress author elements
    contributors = [
        {'id': '', 'login': '', 'email': '', 'display_name': '', 'first_name': '', 'last_name': ''},
        # Add more authors as needed
    ]

    for author_data in contributors:
        author_element = ET.SubElement(channel, 'wp:author')
        for key, value in author_data.items():
            ET.SubElement(author_element, f'wp:{key}').text = value

    # Add WordPress category elements
    categories = [
        {'term_id': '', 'category_nicename': ''},
        # Add more categories as needed
    ]

    for category_data in categories:
        category_element = ET.SubElement(channel, 'wp:category')
        for key, value in category_data.items():
            ET.SubElement(category_element, f'wp:{key}').text = value

    # Add WordPress post elements based on SQLite data
    for row in rows:
        post_element = ET.SubElement(channel, 'item')
        for i, column_name in enumerate(cursor.description):
            ET.SubElement(post_element, column_name[0]).text = str(row[i])

    # Create an ElementTree object and write to an XML file
    tree = ET.ElementTree(root)
    tree.write(output_xml_file, encoding='utf-8', xml_declaration=True)

    # Pretty-print the XML for better readability
    xml_string = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")
    with open(output_xml_file, 'w', encoding='utf-8') as xml_file:
        xml_file.write(xml_string)

    # Close the database connection
    conn.close()

# Example usage
database_file = 'thegate.db'  # Replace with your database file
output_xml_file = 'output2.xml'  # Specify the output XML file
table_name = 'gate_articles'  # Replace with your table name (e.g., 'your_table')

convert_sqlite_to_wordpress_xml(database_file, output_xml_file, table_name)


ExpatError: unbound prefix: line 1, column 224

In [28]:
import sqlite3
import xml.etree.ElementTree as ET
from xml.dom import minidom
from datetime import datetime
import re

def convert_sqlite_to_wordpress_xml(database_file, output_xml_file, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()

    # Execute a query to fetch data from the specified table
    cursor.execute(f'SELECT * FROM {table_name}')
    rows = cursor.fetchall()

    # Create WordPress XML structure
    root = ET.Element('rss', version="2.0")
    root.set('xmlns:excerpt', 'http://wordpress.org/export/1.2/excerpt/')
    root.set('xmlns:content', 'http://purl.org/rss/1.0/modules/content/')
    root.set('xmlns:wfw', 'http://wellformedweb.org/CommentAPI/')
    root.set('xmlns:dc', 'http://purl.org/dc/elements/1.1/')
    root.set('xmlns:wp', 'http://wordpress.org/export/1.2/')
    
    channel = ET.SubElement(root, 'channel')

    for row in rows:
        # Extract data from the row
        article_id, title, contributors, category, image_url, featured_image_caption, lede, body, date = row
        
        # Create WordPress article elements
        item = ET.SubElement(channel, 'item')

        # Title
        title_element = ET.SubElement(item, 'title')
        title_element.text = title

        # Link
        link_element = ET.SubElement(item, 'link')
        link_element.text = f'https://thegateuchicago.wordpress.com.com/{article_id}/'

        # PubDate
        pub_date_element = ET.SubElement(item, 'pubDate')
        try:
            pub_date_element.text = datetime.strptime(date, '%Y-%m-%d %H:%M:%S').strftime('%a, %d %b %Y %H:%M:%S +0000')
        except:
            pub_date_element.text = datetime(2017, 1, 1).strftime('%a, %d %b %Y %H:%M:%S +0000')
        # Creator (Contributors)
        creator_element = ET.SubElement(item, 'dc:creator')
        creator_element.text = contributors

        # GUID
        guid_element = ET.SubElement(item, 'guid', isPermaLink="false")
        guid_element.text = f'https://thegateuchicago.wordpress.com.com/?p={article_id}'

        # Description
        description_element = ET.SubElement(item, 'description')

        # Content
        content_encoded_element = ET.SubElement(item, 'content:encoded')
        content_encoded_element.text = f'<![CDATA[{body}]]>'

        # Excerpt
        excerpt_encoded_element = ET.SubElement(item, 'excerpt:encoded')
        excerpt_encoded_element.text = ''

        # Post ID
        post_id_element = ET.SubElement(item, 'wp:post_id')
        post_id_element.text = str(article_id)

        # Post Date
        post_date_element = ET.SubElement(item, 'wp:post_date')
        post_date_element.text = date

        # Post Date GMT
        post_date_gmt_element = ET.SubElement(item, 'wp:post_date_gmt')
        post_date_gmt_element.text = date

        # Post Modified
        post_modified_element = ET.SubElement(item, 'wp:post_modified')
        post_modified_element.text = date

        # Post Modified GMT
        post_modified_gmt_element = ET.SubElement(item, 'wp:post_modified_gmt')
        post_modified_gmt_element.text = date

        # Comment Status
        comment_status_element = ET.SubElement(item, 'wp:comment_status')
        comment_status_element.text = 'open'

        # Ping Status
        ping_status_element = ET.SubElement(item, 'wp:ping_status')
        ping_status_element.text = 'open'

        # Post Name
        post_name_element = ET.SubElement(item, 'wp:post_name')
        post_name_element.text = f'{title.lower().replace(" ", "-")}-{article_id}'

        # Status
        status_element = ET.SubElement(item, 'wp:status')
        status_element.text = 'publish'

        # Post Parent
        post_parent_element = ET.SubElement(item, 'wp:post_parent')
        post_parent_element.text = '0'

        # Menu Order
        menu_order_element = ET.SubElement(item, 'wp:menu_order')
        menu_order_element.text = '0'

        # Post Type
        post_type_element = ET.SubElement(item, 'wp:post_type')
        post_type_element.text = 'post'

        # Post Password
        post_password_element = ET.SubElement(item, 'wp:post_password')

        # Is Sticky
        is_sticky_element = ET.SubElement(item, 'wp:is_sticky')
        is_sticky_element.text = '0'

        # Categories
        categories = ['category1', 'category2']  # Replace with your categories
        for category_name in categories:
            category_element = ET.SubElement(item, 'category', domain='post_tag', nicename=category_name)
            category_element.text = category_name
        def sanitize_xml_text(text):
            # Remove invalid XML characters
            return re.sub(r'[^\x20-\x7E]', '', text)

        # ... (previous code)

        # Inside the loop where you set text values for XML elements:
        title_element.text = sanitize_xml_text(title)
        creator_element.text = sanitize_xml_text(contributors)
        content_encoded_element.text = sanitize_xml_text(body)


    # Create an ElementTree object and write to an XML file
    tree = ET.ElementTree(root)
    tree.write(output_xml_file, encoding='utf-8', xml_declaration=True)

    # Pretty-print the XML for better readability
    xml_string = minidom.parseString(ET.tostring(root)).toprettyxml(indent="  ")
    with open(output_xml_file, 'w', encoding='utf-8') as xml_file:
        xml_file.write(xml_string)

    # Close the database connection
    conn.close()

# Example usage
database_file = 'thegate.db'  # Replace with your database file
output_xml_file = 'output3.xml'  # Specify the output XML file
table_name = 'gate_articles'  # Replace with your table name

convert_sqlite_to_wordpress_xml(database_file, output_xml_file, table_name)
